# Transformer Encoder

- *Multimodal Transformer for Unaligned Multimodal Language Sequences Yao-Hung Hubert Tsai *, Shaojie Bai *, Paul Pu Liang, J. Zico Kolter, Louis-Philippe Morency, and Ruslan Salakhutdinov Association for Computational Linguistics (ACL), 2019. (*equal contribution)*
- **github link**: https://github.com/yaohungt/Multimodal-Transformer
- original code from *[LINK](https://github.com/yaohungt/Multimodal-Transformer/blob/a670936824ee722c8494fd98d204977a1d663c7a/modules/transformer.py#L9)*

In [6]:
import torch
import math
import torch.nn.functional as F
from torch import nn
import numpy as np

In [10]:
class TransformerEncoder(nn.Module):
    """
    Transformer encoder consisting of *num_layers* layer,
    Each layer is a class of: `TransformerEncoderLayer`.
    Args:
        embed_dim (int)L number of
        num_heads (int): number of heads
        layers (int): number of layers
        attn_dropout (float): dropout applied on the attention weights
        relu_dropout (float): dropout applied on the first layer of the residual block
        res_dropout (float): dropout applied on the residual block
        attn_mask (bool): whether to apply mask on the attention weights
    """
    
    def __init__(self, embed_dim,
                 num_heads,
                 layers,
                 attn_dropout=0.0, 
                 relu_dropout=0.0, 
                 res_dropout=0.0,
                 embed_dropout=0.0,
                 attn_mask=False, 
                 norm=True):
        super(TransformerEncoder, self).__init__()
        self.dropout = embed_dorpout
        self.attn_dropout = attn_dropout
        self.embed_dim = embed_dim
        self.scale = math.sqrt(embed_dim) # scaled dot product
        self.attn_mask = attn_mask
        self.embed_positions = SinusoidalPositionalEmbedding(embed_dim)
        
        # Encoder Layers
        self.layers = nn.ModuleList([])
        for layer in range(layers):
            new_layer = TransformerEncoderLayer(embed_dim,
                                               num_heads=num_heads,
                                               attn_dropout=attn_dropout,
                                               relu_dropout=relu_dropout,
                                               res_dropout=res_dropout,
                                               attn_mask=attn_mask)
            self.layers.append(new_layer)
        
        # Normalization
        self.normalize = norm
        if self.normalize:
            self.layer_norm = LayerNorm(embed_dim)    
            
        self.register_buffer('version', torch.Tensor([2]))
        
        
        
    def forward(self,x, x_in_k, x_in_v):
        """
        Args:
            x_q (FloatTensor): embedded input of shape `(src_len, batch, embed_dim)`
            x_k (FloatTensor): embedded input of shape `(src_len, batch, embed_dim)`
            x_v (FloatTensor): embedded input of shape `(src_len, batch, embed_dim)`
        Returns:
            dict:
                - **encoder_out**
        """
        x = self.embed_scale * x_in # scale
        if self.embed_positions is not None:
            x += self.embed_positions(x_in.transpose(0, 1)[:, :, 0]).transpose(0, 1)   # Add positional embedding
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        if x_in_k is not None and x_in_v is not None:
            # embed tokens and positions
            x_k = self.embed_scale * x_in_k
            x_v = self.embed_scale * x_in_v
            if self.embed_positions is not None:
                x_k += self.embed_positions(x_in_k.transpose(0, 1)[:, :, 0]).transpose(0, 1)   # Add positional embedding
                x_v += self.embed_positions(x_in_v.transpose(0, 1)[:, :, 0]).transpose(0, 1)   # Add positional embedding
            x_k = F.dropout(x_k, p=self.dropout, training=self.training)
            x_v = F.dropout(x_v, p=self.dropout, training=self.training)
        
        # encoder layers
        intermediates = [x]
        for layer in self.layers:
            if x_in_k is not None and x_in_v is not None:
                x = layer(x, x_k, x_v)
            else:
                x = layer(x)
            intermediates.append(x)

        if self.normalize:
            x = self.layer_norm(x)

        return x
    
    
    def max_positions(self):
        """Return Maximum input length supported by the encoder."""
        if self.embed_positions is None:
            return self.max_source_positions
        return min(self.max_source_positions, self.embed_positions.max_positions())

In [19]:
test = torch.tensor([[1,2,3,4],
             [2,4,6,8],
             [1,2,3,4]], dtype=torch.float32)
test = torch.unsqueeze(test, 0).repeat(16, 1, 1).permute(1,0,2)
size = test.shape

print(nn.LayerNorm(size)(test).permute(1,0,2))

tensor([[[-1.1832, -0.6761, -0.1690,  0.3381],
         [-0.6761,  0.3381,  1.3522,  2.3664],
         [-1.1832, -0.6761, -0.1690,  0.3381]],

        [[-1.1832, -0.6761, -0.1690,  0.3381],
         [-0.6761,  0.3381,  1.3522,  2.3664],
         [-1.1832, -0.6761, -0.1690,  0.3381]],

        [[-1.1832, -0.6761, -0.1690,  0.3381],
         [-0.6761,  0.3381,  1.3522,  2.3664],
         [-1.1832, -0.6761, -0.1690,  0.3381]],

        [[-1.1832, -0.6761, -0.1690,  0.3381],
         [-0.6761,  0.3381,  1.3522,  2.3664],
         [-1.1832, -0.6761, -0.1690,  0.3381]],

        [[-1.1832, -0.6761, -0.1690,  0.3381],
         [-0.6761,  0.3381,  1.3522,  2.3664],
         [-1.1832, -0.6761, -0.1690,  0.3381]],

        [[-1.1832, -0.6761, -0.1690,  0.3381],
         [-0.6761,  0.3381,  1.3522,  2.3664],
         [-1.1832, -0.6761, -0.1690,  0.3381]],

        [[-1.1832, -0.6761, -0.1690,  0.3381],
         [-0.6761,  0.3381,  1.3522,  2.3664],
         [-1.1832, -0.6761, -0.1690,  0.3381]],


---
## TransformerEncoderLayer

In [11]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self,embed_dim, num_heads=4, attn_dropout=0.1, relu_dropout=0.1, res_dropout=0.1,
                 attn_mask=False, normalize_before=True, bias=True, kv_bias=False, add_zero_attn=False):
        self.embed_dim = embed_dim
        
        # Multihead Attention 相关
        self.num_heads = num_heads
        self.attn_mask = attn_mask
        self.self_attn = MultiheadAttention(
            embed_dim=self.embed_dim,
            num_heads=self.num_heads,
            attn_dropout=attn_dropout,
            bias=bias,
            kv_bias=kv_bias,
            add_zero_attn=add_zero_attn
        )
        
        self.relu_dropout = relu_dropout
        self.res_dropout = res_dropout
        self.normalize_before = normalize_before # attention 前normalize 或 attention 后 normalize
    
        # The "Add & Norm" part in the paper
        self.fc1 = Linear(self.embed_dim, 4*self.embed_dim)   
        self.fc2 = Linear(4*self.embed_dim, self.embed_dim)
        self.layer_norm1 = LayerNorm(self.embed_dim)
        self.layer_norm2 = LayerNorm(self.embed_dim)

    
    def forward(self, x, x_k=None, x_v=None):
        """
        Args:
            x (Tensor): input to the layer of shape `(batch, seq_len, embed_dim)`
            x_k (Tensor): same as x
            x_v (Tensor): same as x
            attn_mask (ByteTensor): binary ByteTensor of shape
                `(batch, src_len)` where padding elements are indicated by ``1``.
        Returns:
            encoded output of shape `(batch, src_len, embed_dim)`
        """
        residual = x
        # future mask
        if self.attn_mask:
            mask = buffered_future_mask(x, x_k) # (seq_len_q, seq_len_k)
        else:
            mask = None
        
        # layer norm - before attentionx
        x = self.maybe_layer_norm(0, x, before=True)
        
        # masked multihead self attention
        if x_k is None and x_v is None:
            x, _ = self.self_attn(query=x, key=x, value=x, attn_mask=mask)
        else:
            x_k = self.maybe_layer_norm(0, x_k, before=True) # layer norm - before attention
            x_v = self.maybe_layer_norm(0, x_v, before=True) # layer norm - before attention
            x, _ = self.self_attn(query=x, key=x_k, value=x_v, attn_mask=mask)
        
        # Dropout + Add + Norm
        x = F.dropout(x, p=self.res_dropout, training=self.training)
        x = residual + x
        x = self.maybe_layer_norm(0, x, after=True)
        
        # FeedForward + Add + Norm
        residual = x
        x = self.maybe_layer_norm(1, x, before=True)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=self.relu_dropout, training=self.training)
        x = self.fc2(x)
        x = F.dropout(x, p=self.res_dropout, training=self.training)
        x = residual + x
        x = self.maybe_layer_norm(1, x, after=True)    
        return x
    
    
    def maybe_layer_norm(self, i, x, before=False, after=False):
        assert before ^ after
        if after ^ self.normalize_before:
            return self.layer_norms[i](x)
        else:
            return x

            
def buffered_future_mask(tensor, tensor2=None):
    dim1 = dim2 = tensor.size(1)
    if tensor2 is not None:
        dim2 = tensor2.size(1)
    future_mask = torch.triu(torch.full_like(torch.zeros(dim1, dim2), -np.inf), 1+abs(dim2-dim1))
    if tensor.is_cuda:
        future_mask = future_mask.cuda()
    return future_mask[:dim1, :dim2] # (seq_len_1, seq_len_2)        

    
def LayerNorm(embedding_dim):
    m = nn.LayerNorm(embedding_dim)
    return m

---
## Multihead Attention

In [7]:
class MultiheadAttention(nn.Module):
    """
    Multihead-Attention
    "Attention is all you need"
    """
    
    def __init__(self, embed_dim, num_heads, attn_dropout=0., bias=False, add_bias_kv=False, add_zero_attn=False):
        super(MultiheadAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = num_heads // num_heads
        assert self.head_dim * num_heads == self.embed_dim, "embed_dim must be divisible by num_heads"
        self.scale = self.embed_dim ** 0.5 # scale dot product
        self.attn_dropout = attn_dropout
        
        self.in_proj_weight = nn.Parameter(torch.Tensor(3 * embed_dim, embed_dim)) # q, k, v的转换，当q,k,v为同一块内存下的tensor
        self.register_parameter('in_proj_bias', None)
        if bias:
            # q and k and v的转换是否添加bias
            self.in_proj_bias = nn.Parameter(torch.Tensor(3 * embed_dim)) # 对应in_proj_weight的偏置
        self.out_proj = nn.Linear(embed_dim, embed_dim, bias=bias) # bias
        
        if add_bias_kv:
            # k and v的转换是否添加bias
            self.bias_k = Parameter(torch.Tensor(1, 1, embed_dim))
            self.bias_v = Parameter(torch.Tensor(1, 1, embed_dim))
        else:
            self.bias_k = self.bias_v = None
            
        self.add_zero_attn = add_zero_attn
        self.reset_parameters()
        
        
    def reset_parameters(self):
        # 网络参数权重初始化
        nn.init.xavier_uniform_(self.in_proj_weight) # 以xavier方式初始化
        nn.init.xavier_uniform_(self.out_proj.weight)
        
        if self.in_proj_bias is not None:
            nn.init.constant_(self.in_proj_bias, 0.) # 初始化为0
            nn.init.constant_(self.out_proj.bias, 0.)
        if self.bias_k is not None:
            nn.init.xavier_uniform_(self.bias_k)
        if self.bias_v is not None:
            nn.init.xavier_uniform_(self.bias_v)
    
    
    def forward(self, query, key, value, attn_mask=None):
        '''
        Input shape: Input_len(time), Batch, Channel
        Self-attention can be implemented by passing in the same arguments for query, key and value.
        Timesteps can be masked by supplying a T x T mask in the `attn_mask` argument.
        Padding elements can be excluded from the key by passing a binary ByteTensor (`key_padding_mask`) 
            with shape: batch x src_len, where padding elements are indicated by 1s.
        '''
        
        qkv_same = query.data_ptr() == key.data_ptr() == value.data_ptr()
        kv_same = key.data_ptr() == value.data_ptr()
        
        tgt_len, bsz, embed_dim = query.size() # input_len, batch size,  embed_dim (input_len最前面，便于计算q,k,v)
        assert embed_dim == self.embed_dim
        assert list(query.size()) == [tgt_len, bsz, embed_dim]
        assert key.size() == value.size()
        
        aved_state = None #????
        
        if qkv_same:
            # qkv相同，则为self attention
            q,k,v = self.in_proj_qkv(query) # q, k, v为同一块内存下的tensor
        elif kv_same:
            # 只有k,v相同, encoder-decoder attention
            q = self.in_proj_q(query) # 单独对q的权重转换
            if key is None:
                assert value is None # 判断value是否为空，不为空则报错
                k = v = None
            else:
                k, v = self.in_proj_kv(key) # 对k,v一起转换    
        else:
            # q,k,v都不为同一块内存
            q = self.in_proj_q(query)
            k = self.in_proj_k(key)
            v = self.in_proj_v(value)
        q = q * self.scale # Scale
        
        if self.bias_k is not None:
            # k and v转换需要bias
            assert self.bias_v is not None
            k = torch.cat([k, self.bias_k.repeat(1, bsz, 1)])
            v = torch.cat([v, self.bias_v.repeat(1, bsz, 1)])
            if attn_mask is not None:
                attn_mask = torch.cat([attn_mask, attn_mask.new_zeros(attn_mask.size(0), 1)], dim=1)
        
        # 将batch_size转到最前面
        q = q.contiguous().view(tgt_len, bsz * self.num_heads, self.head_dim).transpose(0, 1) 
        if k is not None:
            k = k.contiguous().view(-1, bsz * self.num_heads, self.head_dim).transpose(0,1)
        if v is not None:
            v = k.contiguous().view(-1, bsz * self.num_heads, self.head_dim).transpose(0, 1)
        
        src_len = k.size(1) # seq_len
        
        if self.add_zero_attn:
            # 是否添加zero_attention
            src_len += 1
            k = torch.cat([k, k.new_zeros((k.size(0), 1) + k.size()[2:])], dim=1) #????
            v = torch.cat([v, v.new_zeros((v.size(0), 1) + v.size()[2:])], dim=1)
            if attn_mask is not None:
                attn_mask = torch.cat([attn_mask, attn_mask.new_zeros(attn_mask.size(0), 1)], dim=1)
        
        # dot product attention
        attn_weight = torch.bmm(q, k.transpose(1, 2))
        assert list(attn_weight.size()) == [bsz * self.num_heads, tgt_len, src_len]
        
        if attn_mask is not None:
            try:
                attn_weight += attn_mask.unsqueeze(0) # 在dim=0增加一个维度
            except:
                print(attn_weight.shape)
                print(attn_mask.unsqueeze(0).shape)
                assert False
        
        attn_weights = F.softmax(attn_weights.float(), dim=-1).type_as(attn_weights)
        attn_weights = F.dropout(attn_weights, p=self.attn_dropout, training=self.training)
        
        attn = torch.bmm(attn_weights, v) # apply attention weight to the value
        assert list(attn.size()) == [bsz * self.num_heads, tgt_len, self.head_dim]
        
        # 将多个头合并
        attn = attn.transpose(0, 1).contiguous().view(tgt_len, bsz, embed_dim)
        # 输出时增加linear projectiong
        attn = self.out_proj(attn)
        
        # average attention weights over heads
        attn_weights = attn_weights.view(bsz, self.num_heads, tgt_len, src_len)
        attn_weights = attn_weights.sum(dim=1) / self.num_heads
        return attn, attn_weights
        
        
    
    def _in_proj(self, input, start=0, end=None, **kwargs):
        weight = kwargs.get('weight', self.in_proj_weight) #查找是否有weight参数，没有则使用默认值in_proj_weight
        bias = kwargs.get('bias', self.in_proj_bias) # q or k or v转换时添加bias
        weight = weight[start:end, :] # 将q,k,v转换的对应权重拆开
        
        if bias is not None:
            bias = bias[start:end]
        
        return F.linear(input, weight, bias)
    
    def in_proj_qkv(self, query):
        # q,k,v为同一块内存的tensor,直接转为线性层使用，输出后拆开
        return self._in_proj(query).chunk(3, dim=-1) # 先转换，再分为q,k,v
    
    def in_proj_q(self, query, **kwargs):
        # k,v为同一块内存下的tensor时使用, 对q的转换
        return self._in_proj(query, end=self.embed_dim, **kwargs) # 仅仅拆开q
    
    def in_proj_kv(self, key):
        # k,v为同一块内存下的tensor时使用,对k,v的转换
        return self._in_proj(key, start=self.embed_dim).chunk(2, dim=-1) # 拆开k,v
    
    def in_proj_k(self, key):
        return self._in_proj(key, start=self.embed_dim, end=2 * self.embed_dim)

    def in_proj_v(self, value):
        return self._in_proj(value, start=2 * self.embed_dim)

In [50]:
batch_size = 8
test = torch.tensor([[1,2,3,4],
                     [1,2,3,4], 
                     [1,2,3,4]], dtype=torch.float32)
test = torch.unsqueeze(test, 0).repeat(batch_size, 1, 1)
dim1 = test.shape[1]
dim2 = test.shape[1]
print(test.shape)


print(buffered_future_mask(test.permute(1,0,2)))

torch.triu(torch.full_like(torch.zeros(test.shape[1], test.shape[1]), -np.inf), 1+abs(dim2-dim1))

torch.Size([8, 3, 4])
tensor([[0., -inf, -inf],
        [0., 0., -inf],
        [0., 0., 0.]])


tensor([[0., -inf, -inf],
        [0., 0., -inf],
        [0., 0., 0.]])